In [1]:
import json
import pandas as pd

In [2]:
with open("vns_benchmarks_1h.json", "r", encoding="utf-8") as f:
    vns_1h = json.load(f)

with open("gurobi_benchmarks_1h_verbose.json", "r", encoding="utf-8") as f:
    gurobi_1h = json.load(f)

vns_1h: dict
gurobi_1h: dict

print(vns_1h.keys() == gurobi_1h.keys())

True


In [3]:
project_quantities = [3, 4, 5]
student_quantities = [30, 40, 50]


def get_benchmarking_data_by_dimension(benchmarking_data: dict):
    benchmarking_data_by_dimension = {}
    for project_quantity in project_quantities:
        benchmarking_data_by_dimension[project_quantity] = {}
        for student_quantity in student_quantities:
            benchmarking_data_by_dimension[project_quantity][student_quantity] = {}

    for instance, solutions in benchmarking_data.items():
        for project_quantity in project_quantities:
            for student_quantity in student_quantities:
                if f"_{project_quantity}_{student_quantity}" in instance:
                    benchmarking_data_by_dimension[project_quantity][student_quantity][int(instance[-1])] = solutions
    return benchmarking_data_by_dimension


vns_1h_dimensions = get_benchmarking_data_by_dimension(vns_1h)
gurobi_1h_dimensions = get_benchmarking_data_by_dimension(gurobi_1h)

In [4]:
def round_minor_deviations(x: float) -> float | int:
    nearest = round(x)
    if abs(x - nearest) <= 1e-6:
        return nearest
    return x


def best_solution_within_time_limit(solutions, time_limit=3600):
    for solution in reversed(solutions):
        if solution["runtime"] <= time_limit:
            return solution


def runtime_best_objective_gurobi(solutions, best_objective):
    for solution in solutions:
        if round_minor_deviations(solution["incumbent_obj"]) == best_objective:
            return solution["runtime"]

In [5]:
results_dimensions = {}
for project_quantity in project_quantities:
    results_dimensions[project_quantity] = {}
    for student_quantity in student_quantities:
        vns_1h_dimension = vns_1h_dimensions[project_quantity][student_quantity]
        vns_1h_dimension: dict
        results_dimensions[project_quantity][student_quantity] = {}
        results_dimension = results_dimensions[project_quantity][student_quantity]
        results_dimension["granular"] = pd.DataFrame.from_dict(
            {
                instance: {
                    # "Instance": instance,
                    "Best VNS": (best_vns := int((solution := best_solution_within_time_limit(solutions))["obj"])),
                    "Runtime VNS (s)": float(solution["runtime"]),
                    "Best Gurobi": (
                        best_gurobi := round_minor_deviations(
                            (gurobi_solutions := gurobi_1h_dimensions[project_quantity][student_quantity][instance])[
                                -1
                            ]["incumbent_obj"]
                        )
                    ),
                    "Runtime Gurobi (s)": runtime_best_objective_gurobi(gurobi_solutions, best_gurobi),
                    "Upper Bound": (upper_bound_gurobi := round_minor_deviations(gurobi_solutions[-1]["obj_bound"])),
                    "Gap VNS to Gurobi": (best_gurobi - best_vns) / best_gurobi,
                    "Gap Gurobi": (upper_bound_gurobi - best_gurobi) / upper_bound_gurobi,
                    "Gap VNS": (upper_bound_gurobi - best_vns) / upper_bound_gurobi,
                }
                for instance, solutions in vns_1h_dimension.items()
            },
            orient="index",
        )

        results_dimension_granular = results_dimension["granular"]
        results_dimension_granular: pd.DataFrame

        means = results_dimension_granular[
            ["Runtime VNS (s)", "Runtime Gurobi (s)", "Gap VNS to Gurobi", "Gap Gurobi", "Gap VNS"]
        ].mean()
        medians = results_dimension_granular[
            ["Runtime VNS (s)", "Runtime Gurobi (s)", "Gap VNS to Gurobi", "Gap Gurobi", "Gap VNS"]
        ].median()
        maxima = results_dimension_granular[
            ["Runtime VNS (s)", "Runtime Gurobi (s)", "Gap VNS to Gurobi", "Gap Gurobi", "Gap VNS"]
        ].max()
        minima = results_dimension_granular[
            ["Runtime VNS (s)", "Runtime Gurobi (s)", "Gap VNS to Gurobi", "Gap Gurobi", "Gap VNS"]
        ].min()
        results_dimension["summary"] = pd.DataFrame(
            [means, medians, maxima, minima], index=["Mean", "Median", "Maximum", "Minimum"]
        )
        results_dimension_summary = results_dimension["summary"]
        results_dimension_summary: pd.DataFrame

        results_dimension_summary_presentation = results_dimension_summary.copy()
        results_dimension_summary_presentation[["Gap VNS to Gurobi", "Gap Gurobi", "Gap VNS"]] = (
            results_dimension_summary_presentation[["Gap VNS to Gurobi", "Gap Gurobi", "Gap VNS"]].map("{:.2%}".format)
        )
        results_dimension_summary_presentation[["Runtime VNS (s)", "Runtime Gurobi (s)"]] = (
            results_dimension_summary_presentation[["Runtime VNS (s)", "Runtime Gurobi (s)"]].map("{:.2f}".format)
        )
        results_dimension["summary_presentation"] = results_dimension_summary_presentation

        results_dimension_presentation = results_dimension_granular.copy()
        results_dimension_presentation[["Best VNS", "Best Gurobi", "Upper Bound"]] = results_dimension_presentation[
            ["Best VNS", "Best Gurobi", "Upper Bound"]
        ].map("{:.0f}".format)
        results_dimension_presentation[["Gap VNS to Gurobi", "Gap Gurobi", "Gap VNS"]] = (
            results_dimension_presentation[["Gap VNS to Gurobi", "Gap Gurobi", "Gap VNS"]].map("{:.2%}".format)
        )
        results_dimension_presentation[["Runtime VNS (s)", "Runtime Gurobi (s)"]] = results_dimension_presentation[
            ["Runtime VNS (s)", "Runtime Gurobi (s)"]
        ].map("{:.2f}".format)
        results_dimension_presentation.index.name = "Instance"
        results_dimension["granular_presentation"] = results_dimension_presentation

In [6]:
results_dimensions[3][30]["granular_presentation"]


,Best VNS,Runtime VNS (s),Best Gurobi,Runtime Gurobi (s),Upper Bound,Gap VNS to Gurobi,Gap Gurobi,Gap VNS
Instance,,,,,,,,
0,79,1.66,81,0.44,81,2.47%,0.00%,2.47%
1,92,4.62,92,0.60,92,0.00%,0.00%,0.00%
2,86,13.89,86,1.45,86,0.00%,0.00%,0.00%
3,95,43.71,98,0.47,98,3.06%,0.00%,3.06%
4,100,377.74,101,2.19,101,0.99%,0.00%,0.99%
5,96,9.15,96,1.03,96,0.00%,0.00%,0.00%
6,98,185.42,98,0.23,98,0.00%,0.00%,0.00%
7,69,10.45,71,1.61,71,2.82%,0.00%,2.82%
8,100,104.30,101,0.70,101,0.99%,0.00%,0.99%


In [7]:
results_dimensions[3][30]["summary_presentation"]

,Runtime VNS (s),Runtime Gurobi (s),Gap VNS to Gurobi,Gap Gurobi,Gap VNS
Mean,75.59,0.97,1.16%,0.00%,1.16%
Median,12.17,0.84,0.99%,0.00%,0.99%
Maximum,377.74,2.19,3.06%,0.00%,3.06%
Minimum,1.66,0.23,0.00%,0.00%,0.00%


In [8]:
results_dimensions[3][40]["granular_presentation"]

,Best VNS,Runtime VNS (s),Best Gurobi,Runtime Gurobi (s),Upper Bound,Gap VNS to Gurobi,Gap Gurobi,Gap VNS
Instance,,,,,,,,
0,118,698.81,118,2.03,118,0.00%,0.00%,0.00%
1,108,1723.24,112,0.66,112,3.57%,0.00%,3.57%
2,114,7.26,116,65.29,116,1.72%,0.00%,1.72%
3,113,12.75,114,0.50,114,0.88%,0.00%,0.88%
4,121,106.73,123,6.19,127,1.63%,3.15%,4.72%
5,119,533.79,122,2.95,122,2.46%,0.00%,2.46%
6,102,47.55,102,1.38,102,0.00%,0.00%,0.00%
7,105,752.09,105,1.13,105,0.00%,0.00%,0.00%
8,119,21.71,121,3.67,121,1.65%,0.00%,1.65%


In [9]:
results_dimensions[3][40]["summary_presentation"]

,Runtime VNS (s),Runtime Gurobi (s),Gap VNS to Gurobi,Gap Gurobi,Gap VNS
Mean,390.69,8.56,1.19%,0.31%,1.50%
Median,77.14,1.94,1.25%,0.00%,1.27%
Maximum,1723.24,65.29,3.57%,3.15%,4.72%
Minimum,2.93,0.50,0.00%,0.00%,0.00%


In [10]:
results_dimensions[3][50]["granular_presentation"]

,Best VNS,Runtime VNS (s),Best Gurobi,Runtime Gurobi (s),Upper Bound,Gap VNS to Gurobi,Gap Gurobi,Gap VNS
Instance,,,,,,,,
0,157,1223.21,160,5.63,160,1.88%,0.00%,1.88%
1,151,81.65,154,1.06,154,1.95%,0.00%,1.95%
2,139,2688.54,139,131.65,139,0.00%,0.00%,0.00%
3,121,2927.25,124,18.87,126,2.42%,1.59%,3.97%
4,87,3112.12,88,13.91,88,1.14%,0.00%,1.14%
5,145,1392.83,148,586.86,153,2.03%,3.27%,5.23%
6,131,260.04,133,23.20,133,1.50%,0.00%,1.50%
7,153,3398.76,154,1.73,157,0.65%,1.91%,2.55%
8,131,747.72,136,13.89,136,3.68%,0.00%,3.68%


In [11]:
results_dimensions[3][50]["summary_presentation"]

,Runtime VNS (s),Runtime Gurobi (s),Gap VNS to Gurobi,Gap Gurobi,Gap VNS
Mean,1699.09,96.10,1.68%,1.11%,2.77%
Median,1308.02,16.39,1.70%,0.00%,2.25%
Maximum,3398.76,586.86,3.68%,4.38%,5.84%
Minimum,81.65,1.06,0.00%,0.00%,0.00%


In [12]:
results_dimensions[4][30]["granular_presentation"]

,Best VNS,Runtime VNS (s),Best Gurobi,Runtime Gurobi (s),Upper Bound,Gap VNS to Gurobi,Gap Gurobi,Gap VNS
Instance,,,,,,,,
0,89,121.26,89,1.09,89,0.00%,0.00%,0.00%
1,92,41.01,96,0.73,96,4.17%,0.00%,4.17%
2,103,69.14,103,103.94,103,0.00%,0.00%,0.00%
3,103,12.06,104,0.55,104,0.96%,0.00%,0.96%
4,109,3057.09,111,2.03,111,1.80%,0.00%,1.80%
5,88,64.15,89,0.76,89,1.12%,0.00%,1.12%
6,111,202.35,114,3.70,114,2.63%,0.00%,2.63%
7,90,36.61,95,2.20,95,5.26%,0.00%,5.26%
8,84,1.63,87,0.87,87,3.45%,0.00%,3.45%


In [13]:
results_dimensions[4][30]["summary_presentation"]

,Runtime VNS (s),Runtime Gurobi (s),Gap VNS to Gurobi,Gap Gurobi,Gap VNS
Mean,360.63,11.95,2.46%,0.00%,2.46%
Median,52.58,1.56,2.22%,0.00%,2.22%
Maximum,3057.09,103.94,5.26%,0.00%,5.26%
Minimum,1.01,0.55,0.00%,0.00%,0.00%


In [14]:
results_dimensions[4][40]["granular_presentation"]

,Best VNS,Runtime VNS (s),Best Gurobi,Runtime Gurobi (s),Upper Bound,Gap VNS to Gurobi,Gap Gurobi,Gap VNS
Instance,,,,,,,,
0,128,193.28,129,0.52,129,0.78%,0.00%,0.78%
1,118,1313.62,118,0.93,121,0.00%,2.48%,2.48%
2,124,4.91,124,215.13,130,0.00%,4.62%,4.62%
3,115,149.09,117,19.41,117,1.71%,0.00%,1.71%
4,115,435.82,116,0.46,116,0.86%,0.00%,0.86%
5,117,943.57,119,13.16,122,1.68%,2.46%,4.10%
6,120,510.66,121,52.15,121,0.83%,0.00%,0.83%
7,123,395.80,126,2.11,126,2.38%,0.00%,2.38%
8,135,2364.03,135,7.24,135,0.00%,0.00%,0.00%


In [15]:
results_dimensions[4][40]["summary_presentation"]

,Runtime VNS (s),Runtime Gurobi (s),Gap VNS to Gurobi,Gap Gurobi,Gap VNS
Mean,716.89,33.65,0.97%,0.96%,1.92%
Median,473.24,10.20,0.84%,0.00%,1.60%
Maximum,2364.03,215.13,2.38%,4.62%,4.62%
Minimum,4.91,0.46,0.00%,0.00%,0.00%


In [16]:
results_dimensions[4][50]["granular_presentation"]

,Best VNS,Runtime VNS (s),Best Gurobi,Runtime Gurobi (s),Upper Bound,Gap VNS to Gurobi,Gap Gurobi,Gap VNS
Instance,,,,,,,,
0,153,96.26,159,32.44,159,3.77%,0.00%,3.77%
1,146,2423.84,148,8.51,152,1.35%,2.63%,3.95%
2,159,419.55,160,8.10,160,0.62%,0.00%,0.62%
3,148,295.89,150,7.10,154,1.33%,2.60%,3.90%
4,168,42.61,170,5.07,170,1.18%,0.00%,1.18%
5,149,725.45,153,1.69,153,2.61%,0.00%,2.61%
6,155,2402.07,157,4.00,165,1.27%,4.85%,6.06%
7,155,538.24,157,68.42,157,1.27%,0.00%,1.27%
8,155,250.93,161,2321.30,168,3.73%,4.17%,7.74%


In [17]:
results_dimensions[4][50]["summary_presentation"]

,Runtime VNS (s),Runtime Gurobi (s),Gap VNS to Gurobi,Gap Gurobi,Gap VNS
Mean,741.88,252.25,1.92%,1.42%,3.32%
Median,357.72,8.30,1.34%,0.00%,3.19%
Maximum,2423.84,2321.30,3.77%,4.85%,7.74%
Minimum,42.61,1.69,0.62%,0.00%,0.62%


In [18]:
results_dimensions[5][30]["granular_presentation"]

,Best VNS,Runtime VNS (s),Best Gurobi,Runtime Gurobi (s),Upper Bound,Gap VNS to Gurobi,Gap Gurobi,Gap VNS
Instance,,,,,,,,
0,94,1.79,97,12.78,97,3.09%,0.00%,3.09%
1,92,3428.53,94,1.60,94,2.13%,0.00%,2.13%
2,100,50.25,102,1.03,102,1.96%,0.00%,1.96%
3,107,31.41,110,1.35,110,2.73%,0.00%,2.73%
4,92,56.14,103,7.40,103,10.68%,0.00%,10.68%
5,98,147.46,100,11.50,100,2.00%,0.00%,2.00%
6,84,51.23,87,2.96,87,3.45%,0.00%,3.45%
7,92,118.72,94,9.83,94,2.13%,0.00%,2.13%
8,103,226.29,103,3.13,103,0.00%,0.00%,0.00%


In [19]:
results_dimensions[5][30]["summary_presentation"]

,Runtime VNS (s),Runtime Gurobi (s),Gap VNS to Gurobi,Gap Gurobi,Gap VNS
Mean,412.87,5.28,3.03%,0.00%,3.03%
Median,53.68,3.05,2.14%,0.00%,2.14%
Maximum,3428.53,12.78,10.68%,0.00%,10.68%
Minimum,1.79,1.03,0.00%,0.00%,0.00%


In [20]:
results_dimensions[5][40]["granular_presentation"]

,Best VNS,Runtime VNS (s),Best Gurobi,Runtime Gurobi (s),Upper Bound,Gap VNS to Gurobi,Gap Gurobi,Gap VNS
Instance,,,,,,,,
0,133,497.28,138,2.76,139,3.62%,0.72%,4.32%
1,133,18.06,137,2.65,137,2.92%,0.00%,2.92%
2,134,271.22,137,96.18,137,2.19%,0.00%,2.19%
3,132,10.61,135,2.00,135,2.22%,0.00%,2.22%
4,128,3069.66,128,12.41,128,0.00%,0.00%,0.00%
5,130,9.06,132,62.17,132,1.52%,0.00%,1.52%
6,134,203.88,138,21.71,138,2.90%,0.00%,2.90%
7,131,2894.73,134,9.03,134,2.24%,0.00%,2.24%
8,129,47.32,130,4.89,130,0.77%,0.00%,0.77%


In [21]:
results_dimensions[5][40]["summary_presentation"]

,Runtime VNS (s),Runtime Gurobi (s),Gap VNS to Gurobi,Gap Gurobi,Gap VNS
Mean,713.42,27.42,2.37%,0.07%,2.44%
Median,158.13,10.72,2.23%,0.00%,2.23%
Maximum,3069.66,96.18,5.31%,0.72%,5.31%
Minimum,9.06,2.00,0.00%,0.00%,0.00%


In [22]:
results_dimensions[5][50]["granular_presentation"]

,Best VNS,Runtime VNS (s),Best Gurobi,Runtime Gurobi (s),Upper Bound,Gap VNS to Gurobi,Gap Gurobi,Gap VNS
Instance,,,,,,,,
0,167,1210.18,170,119.61,177,1.76%,3.95%,5.65%
1,146,400.80,153,591.76,162,4.58%,5.56%,9.88%
2,170,1027.83,170,4.77,178,0.00%,4.49%,4.49%
3,166,20.81,169,14.48,172,1.78%,1.74%,3.49%
4,162,131.50,167,16.94,169,2.99%,1.18%,4.14%
5,158,37.69,162,497.14,165,2.47%,1.82%,4.24%
6,150,357.92,152,15.58,161,1.32%,5.59%,6.83%
7,145,1049.44,147,6.13,147,1.36%,0.00%,1.36%
8,168,17.76,171,98.55,171,1.75%,0.00%,1.75%


In [23]:
results_dimensions[5][50]["summary_presentation"]

,Runtime VNS (s),Runtime Gurobi (s),Gap VNS to Gurobi,Gap Gurobi,Gap VNS
Mean,426.95,412.69,2.10%,3.32%,5.35%
Median,244.71,57.74,1.77%,2.89%,4.37%
Maximum,1210.18,2761.94,4.58%,8.90%,11.64%
Minimum,15.58,4.77,0.00%,0.00%,1.36%


In [24]:
results_all_instances = pd.DataFrame.from_dict(
    {
        instance: {
            "Best VNS": (best_vns := int((solution := best_solution_within_time_limit(solutions))["obj"])),
            "Runtime VNS (s)": solution["runtime"],
            "Best Gurobi": (
                best_gurobi := round_minor_deviations((gurobi_solutions := gurobi_1h[instance])[-1]["incumbent_obj"])
            ),
            "Runtime Gurobi (s)": runtime_best_objective_gurobi(gurobi_solutions, best_gurobi),
            "Upper Bound": (upper_bound_gurobi := round_minor_deviations(gurobi_solutions[-1]["obj_bound"])),
            "Gap VNS to Gurobi": (best_gurobi - best_vns) / best_gurobi,
            "Gap Gurobi": (upper_bound_gurobi - best_gurobi) / upper_bound_gurobi,
            "Gap VNS": (upper_bound_gurobi - best_vns) / upper_bound_gurobi,
        }
        for instance, solutions in vns_1h.items()
    },
    orient="index",
)

means = results_all_instances[
    ["Runtime VNS (s)", "Runtime Gurobi (s)", "Gap VNS to Gurobi", "Gap Gurobi", "Gap VNS"]
].mean()

medians = results_all_instances[
    ["Runtime VNS (s)", "Runtime Gurobi (s)", "Gap VNS to Gurobi", "Gap Gurobi", "Gap VNS"]
].median()

maxima = results_all_instances[
    ["Runtime VNS (s)", "Runtime Gurobi (s)", "Gap VNS to Gurobi", "Gap Gurobi", "Gap VNS"]
].max()

minima = results_all_instances[
    ["Runtime VNS (s)", "Runtime Gurobi (s)", "Gap VNS to Gurobi", "Gap Gurobi", "Gap VNS"]
].min()

summary_stats_all_instances = pd.DataFrame(
    [means, medians, maxima, minima], index=["Mean", "Median", "Maximum", "Minimum"]
)

summary_stats_all_instances_presentation = summary_stats_all_instances.copy()
summary_stats_all_instances_presentation[["Gap VNS to Gurobi", "Gap Gurobi", "Gap VNS"]] = (
    summary_stats_all_instances_presentation[["Gap VNS to Gurobi", "Gap Gurobi", "Gap VNS"]].map("{:.2%}".format)
)
summary_stats_all_instances_presentation[["Runtime VNS (s)", "Runtime Gurobi (s)"]] = (
    summary_stats_all_instances_presentation[["Runtime VNS (s)", "Runtime Gurobi (s)"]].map("{:.2f}".format)
)

In [25]:
summary_stats_all_instances_presentation

,Runtime VNS (s),Runtime Gurobi (s),Gap VNS to Gurobi,Gap Gurobi,Gap VNS
Mean,615.33,94.32,1.87%,0.80%,2.66%
Median,167.25,4.83,1.72%,0.00%,2.17%
Maximum,3428.53,2761.94,10.68%,8.90%,11.64%
Minimum,1.01,0.23,0.00%,0.00%,0.00%


In [26]:
# summary_stats_all_instances_presentation.to_csv("summary_all_instances.csv", sep=";")

In [31]:
print(
    results_dimensions[5][50]["granular_presentation"].to_latex(
        index=True,
        caption="3 Projects, 30 Students",
        label="tab:3p30s",
        float_format="%.2f".format,
        column_format="lrrrrrrrr",
        escape=True,
        position="htbp",
    )
)

\begin{table}[htbp]
\caption{3 Projects, 30 Students}
\label{tab:3p30s}
\begin{tabular}{lrrrrrrrr}
\toprule
 & Best VNS & Runtime VNS (s) & Best Gurobi & Runtime Gurobi (s) & Upper Bound & Gap VNS to Gurobi & Gap Gurobi & Gap VNS \\
Instance &  &  &  &  &  &  &  &  \\
\midrule
0 & 167 & 1210.18 & 170 & 119.61 & 177 & 1.76\% & 3.95\% & 5.65\% \\
1 & 146 & 400.80 & 153 & 591.76 & 162 & 4.58\% & 5.56\% & 9.88\% \\
2 & 170 & 1027.83 & 170 & 4.77 & 178 & 0.00\% & 4.49\% & 4.49\% \\
3 & 166 & 20.81 & 169 & 14.48 & 172 & 1.78\% & 1.74\% & 3.49\% \\
4 & 162 & 131.50 & 167 & 16.94 & 169 & 2.99\% & 1.18\% & 4.14\% \\
5 & 158 & 37.69 & 162 & 497.14 & 165 & 2.47\% & 1.82\% & 4.24\% \\
6 & 150 & 357.92 & 152 & 15.58 & 161 & 1.32\% & 5.59\% & 6.83\% \\
7 & 145 & 1049.44 & 147 & 6.13 & 147 & 1.36\% & 0.00\% & 1.36\% \\
8 & 168 & 17.76 & 171 & 98.55 & 171 & 1.75\% & 0.00\% & 1.75\% \\
9 & 129 & 15.58 & 133 & 2761.94 & 146 & 3.01\% & 8.90\% & 11.64\% \\
\bottomrule
\end{tabular}
\end{table}



In [51]:
print(
    results_dimensions[5][50]["granular_presentation"].to_latex(
        index=True,
        caption="3 Projects, 30 Students",
        label="tab:3p30s",
        float_format="%.2f".format,
        column_format="lrrrrrrrr",
        escape=True,
        position="htbp",
    )
)

\begin{table}[htbp]
\caption{3 Projects, 30 Students}
\label{tab:3p30s}
\begin{tabular}{lrrrrrrrr}
\toprule
 & Best VNS & Runtime VNS (s) & Best Gurobi & Runtime Gurobi (s) & Upper Bound & Gap VNS to Gurobi & Gap Gurobi & Gap VNS \\
Instance &  &  &  &  &  &  &  &  \\
\midrule
0 & 167 & 1210.18 & 170 & 119.61 & 177 & 1.76\% & 3.95\% & 5.65\% \\
1 & 146 & 400.80 & 153 & 591.76 & 162 & 4.58\% & 5.56\% & 9.88\% \\
2 & 170 & 1027.83 & 170 & 4.77 & 178 & 0.00\% & 4.49\% & 4.49\% \\
3 & 166 & 20.81 & 169 & 14.48 & 172 & 1.78\% & 1.74\% & 3.49\% \\
4 & 162 & 131.50 & 167 & 16.94 & 169 & 2.99\% & 1.18\% & 4.14\% \\
5 & 158 & 37.69 & 162 & 497.14 & 165 & 2.47\% & 1.82\% & 4.24\% \\
6 & 150 & 357.92 & 152 & 15.58 & 161 & 1.32\% & 5.59\% & 6.83\% \\
7 & 145 & 1049.44 & 147 & 6.13 & 147 & 1.36\% & 0.00\% & 1.36\% \\
8 & 168 & 17.76 & 171 & 98.55 & 171 & 1.75\% & 0.00\% & 1.75\% \\
9 & 129 & 15.58 & 133 & 2761.94 & 146 & 3.01\% & 8.90\% & 11.64\% \\
\bottomrule
\end{tabular}
\end{table}



In [52]:
print(
    results_dimensions[5][50]["summary_presentation"].to_latex(
        index=True,
        caption="3 Projects, 30 Students",
        label="tab:3p30s",
        float_format="%.2f".format,
        column_format="lrrrrrrrr",
        escape=True,
        position="htbp",
    )
)

\begin{table}[htbp]
\caption{3 Projects, 30 Students}
\label{tab:3p30s}
\begin{tabular}{lrrrrrrrr}
\toprule
 & Runtime VNS (s) & Runtime Gurobi (s) & Gap VNS to Gurobi & Gap Gurobi & Gap VNS \\
\midrule
Mean & 426.95 & 412.69 & 2.10\% & 3.32\% & 5.35\% \\
Median & 244.71 & 57.74 & 1.77\% & 2.89\% & 4.37\% \\
Maximum & 1210.18 & 2761.94 & 4.58\% & 8.90\% & 11.64\% \\
Minimum & 15.58 & 4.77 & 0.00\% & 0.00\% & 1.36\% \\
\bottomrule
\end{tabular}
\end{table}

